In [18]:
import requests
from dotenv import load_dotenv

In [19]:
# GET API KEY FROM THE .env file
load_dotenv()
API_KEY = os.getenv('API_KEY')

### gettime request - checks for local clock synced with CTA

In [20]:
api_url = f'http://www.ctabustracker.com/bustime/api/v2/gettime?key={API_KEY}'

response = requests.get(api_url)
response.headers["Content-Type"]
response.text

'<?xml version="1.0"?>\r\n<bustime-response><tm>20230108 01:14:16</tm></bustime-response>\r\n'

### getvehicles request by route(s) or vehicle(s)

In [21]:
# Must specify a route or list of routes via &rt=20,22 (to get routes 20 and 22)
# Alternatively, you could specify a vehicle or list of vehicles instead
# Must specify &format=json, otherwise format will be html

api_url = f'http://www.ctabustracker.com/bustime/api/v2/getvehicles?key={API_KEY}&rt=20,22&format=json'

response = requests.get(api_url)
test_vehicles_20_and_22 = response.json()
test_vehicles_20_and_22

{'bustime-response': {'vehicle': [{'vid': '8124',
    'tmstmp': '20230108 01:13',
    'lat': '41.882146666666664',
    'lon': '-87.62783',
    'hdg': '266',
    'pid': 947,
    'rt': '20',
    'des': 'Austin',
    'pdist': 1988,
    'dly': False,
    'tatripid': '1040772',
    'origtatripno': '238683358',
    'tablockid': '20 -808',
    'zone': ''},
   {'vid': '8167',
    'tmstmp': '20230108 01:13',
    'lat': '41.88128502689191',
    'lon': '-87.68604563243353',
    'hdg': '269',
    'pid': 954,
    'rt': '20',
    'des': 'Austin',
    'pdist': 16782,
    'dly': False,
    'tatripid': '1040767',
    'origtatripno': '235317487',
    'tablockid': 'N20 -891',
    'zone': ''},
   {'vid': '8112',
    'tmstmp': '20230108 01:13',
    'lat': '41.88046833333333',
    'lon': '-87.77395666666666',
    'hdg': '179',
    'pid': 954,
    'rt': '20',
    'des': 'Austin',
    'pdist': 41341,
    'dly': True,
    'tatripid': '1040767',
    'origtatripno': '238685858',
    'tablockid': 'N20 -891',
    

### getroutes request

In [22]:
# Must specify &format=json, otherwise format will be html

api_url = f'http://www.ctabustracker.com/bustime/api/v2/getroutes?key={API_KEY}&format=json'

response = requests.get(api_url)
routes = response.json()
routes

{'bustime-response': {'routes': [{'rt': '1',
    'rtnm': 'Bronzeville/Union Station',
    'rtclr': '#336633',
    'rtdd': '1'},
   {'rt': '2', 'rtnm': 'Hyde Park Express', 'rtclr': '#993366', 'rtdd': '2'},
   {'rt': '3', 'rtnm': 'King Drive', 'rtclr': '#009900', 'rtdd': '3'},
   {'rt': '4', 'rtnm': 'Cottage Grove', 'rtclr': '#cc3300', 'rtdd': '4'},
   {'rt': 'X4',
    'rtnm': 'Cottage Grove Express',
    'rtclr': '#006666',
    'rtdd': 'X4'},
   {'rt': '5',
    'rtnm': 'South Shore Night Bus',
    'rtclr': '#996633',
    'rtdd': '5'},
   {'rt': '6',
    'rtnm': 'Jackson Park Express',
    'rtclr': '#ff0066',
    'rtdd': '6'},
   {'rt': '7', 'rtnm': 'Harrison', 'rtclr': '#666600', 'rtdd': '7'},
   {'rt': '8', 'rtnm': 'Halsted', 'rtclr': '#ff00ff', 'rtdd': '8'},
   {'rt': '8A', 'rtnm': 'South Halsted', 'rtclr': '#66cc66', 'rtdd': '8A'},
   {'rt': '9', 'rtnm': 'Ashland', 'rtclr': '#cc3366', 'rtdd': '9'},
   {'rt': 'X9', 'rtnm': 'Ashland Express', 'rtclr': '#33ff00', 'rtdd': 'X9'},
   {'rt

### getdirections request

In [23]:
# Must specify a single route: &rt=20 for Route 20, for example

api_url = f'http://www.ctabustracker.com/bustime/api/v2/getdirections?key={API_KEY}&rt=20&format=json'

response = requests.get(api_url)
directions_rt_20 = response.json()
directions_rt_20

{'bustime-response': {'directions': [{'dir': 'Eastbound'},
   {'dir': 'Westbound'}]}}

### getstops request

In [29]:
# Route and direction are both required

api_url = f'http://www.ctabustracker.com/bustime/api/v2/getstops?key={API_KEY}&rt=20&dir=Westbound&format=json'

response = requests.get(api_url)
stops_rt20_wb = response.json()
stops_rt20_wb

{'bustime-response': {'stops': [{'stpid': '8613',
    'stpnm': 'Columbus & Randolph',
    'lat': 41.884753999999425,
    'lon': -87.6207790000014},
   {'stpid': '460',
    'stpnm': 'Madison & Aberdeen ',
    'lat': 41.88178999999918,
    'lon': -87.65442299999938},
   {'stpid': '480',
    'stpnm': 'Madison & Albany ',
    'lat': 41.88101799999979,
    'lon': -87.70341200000043},
   {'stpid': '465',
    'stpnm': 'Madison & Ashland',
    'lat': 41.88152000000094,
    'lon': -87.66669500000171},
   {'stpid': '14800',
    'stpnm': 'Madison & Austin',
    'lat': 41.880191999999454,
    'lon': -87.7743480000017},
   {'stpid': '386',
    'stpnm': 'Madison & Austin Terminal',
    'lat': 41.880294999999315,
    'lon': -87.77407799999872},
   {'stpid': '14841',
    'stpnm': 'Madison & California',
    'lat': 41.88113799999881,
    'lon': -87.6967350000011},
   {'stpid': '16087',
    'stpnm': 'Madison & Campbell',
    'lat': 41.88120199999954,
    'lon': -87.68888300000053},
   {'stpid': '505',
 